In [ ]:
import os
import shutil

import papermill as pm

In [ ]:
username = "Fro116"
source = "MAL"
# username = "33"
# source = "Training"

In [ ]:
data_path = os.path.abspath("../../data/recommendations")
if not os.path.exists(data_path):
    os.mkdir(data_path)
rec_dir = os.path.join(data_path, username)
papermill_path = os.path.join(rec_dir, "papermill")

## RecIngress

In [ ]:
def run_ingress_notebook(name):
    outdir = os.path.join(papermill_path, "rec_ingress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    pm.execute_notebook(
        f"../RecIngress/{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source),
    )

In [ ]:
if os.path.exists(rec_dir):
    shutil.rmtree(rec_dir)

In [ ]:
run_ingress_notebook("FetchAnimeLists")

In [ ]:
run_ingress_notebook("ImportAnimeLists")

In [ ]:
run_ingress_notebook("ProcessAnimeLists")

In [ ]:
run_ingress_notebook("RelabelIds")

In [ ]:
run_ingress_notebook("GenerateSplits")

In [ ]:
run_ingress_notebook("CompressSplits")

## InferenceAlphas

In [ ]:
def run_inference_alphas_notebook(name):
    cwd = os.getcwd()
    os.chdir("../InferenceAlphas")
    for folder in name.split("/")[:-1]:
        os.chdir(folder)
    try:
        outdir = os.path.join(papermill_path, "inference_alphas")
        for folder in name.split("/")[:-1]:
            outdir = os.path.join(outdir, folder.lower())
        basename = name.split("/")[-1]
        if not os.path.exists(outdir):
            os.makedirs(outdir, exist_ok=True)
        pm.execute_notebook(
            f"{basename}.ipynb",
            f"{outdir}/{basename}.ipynb",
            parameters=dict(username=username, source=source, outdir=outdir),
        )
    finally:
        os.chdir(cwd)

In [ ]:
run_inference_alphas_notebook("ResetAlphas")

In [ ]:
run_inference_alphas_notebook("Simple/RunAllScripts")

In [ ]:
run_inference_alphas_notebook("Explicit/RunAllScripts")

In [ ]:
run_inference_alphas_notebook("Nondirectional/RunAllScripts")

In [ ]:
run_inference_alphas_notebook("Neural/RunAllScripts")

In [ ]:
run_inference_alphas_notebook("Ensemble/RunAllScripts")

## RecEgress

In [ ]:
def run_egress_notebook(name):
    outdir = os.path.join(papermill_path, "rec_egress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    pm.execute_notebook(
        f"{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source),
    )

In [ ]:
if source == "Training":
    run_egress_notebook("TrainingServingSkew")
else:
    run_egress_notebook("Recommendations")

In [ ]:
print("Success!")